In [176]:
# this block imports all necessary libraries and a necessary thing is for this code to work, run ganache 
# and replace the address and publickeys as are required.

from eth_utils import address
from web3 import Web3
import os
import solcx
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json


In [177]:
# opening the SCR contract file

with open("./SCR.sol", "r") as file:
    registrationFile = file.read()


In [178]:
#compiling smart contract of certifier(SCR)
install_solc("0.8.2")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCR.sol": {"content": registrationFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [179]:
# get bytecode
bytecode = compiled_sol["contracts"]["SCR.sol"]["Registration"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(
    compiled_sol["contracts"]["SCR.sol"]["Registration"]["metadata"]
)["output"]["abi"]

In [180]:
# Opening the SCA contract File
    
with open("./SCA.sol", "r") as file:
    AuctioneerFile = file.read()

In [181]:
# compiling SmartContract of auctioneer(SCA)
install_solc("0.8.2")
compiled_solA = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCA.sol": {"content": AuctioneerFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_solA, file)

In [182]:
# get bytecode
bytecodeA = compiled_solA["contracts"]["SCA.sol"]["Submission"]["evm"]["bytecode"]["object"]

# get abi
abiA = json.loads(
    compiled_solA["contracts"]["SCA.sol"]["Submission"]["metadata"]
)["output"]["abi"]

In [183]:
x = abi
k = abiA

In [184]:
abiA

[{'inputs': [{'internalType': 'string', 'name': 'slices', 'type': 'string'}],
  'name': 'bidderEncryptedBidSubmission',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'fetchVariable',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'retrieveBidderEncryptedBidSubmission',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'string',
    'name': 'auctioneerPB',
    'type': 'string'}],
  'name': 'updateBidderVariable',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [185]:
abi = abi[1]
abiA = abiA[0],abiA[3]

In [186]:
abiA

({'inputs': [{'internalType': 'string', 'name': 'slices', 'type': 'string'}],
  'name': 'bidderEncryptedBidSubmission',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'string',
    'name': 'auctioneerPB',
    'type': 'string'}],
  'name': 'updateBidderVariable',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'})

In [187]:
chainID = 1337
chain_id =chainID
httpServer = "HTTP://127.0.0.1:7545"
w3 = Web3(Web3.HTTPProvider(httpServer))

In [188]:


# set up connection for bidder

my_addressB = '0x315f377E0B0D4aB7545dCB5e24781a9360Cc4055'
private_keyB = '0xf950554f1d5102f9e66efb03cdbcfef26f68b6af9986196d3ecda087557102a8'

# set up connection for certifier

my_addressC = '0xDb7C23c1934eD2991D209FfdFb7a94315BfE181D'
private_keyC = '0xcb8a91ea1525a7d698bc366fdd0885f4c0ca7fe08b474ec2c3079df478cdd4f9'

# set up connection for auctioneer
my_addressA = '0xbe554a91C2e7015Ede9725eCe41228ff8b7fBC98'
private_keyA = "0x66c4970cbd4595cf0b57ca9de0d64a6c63e76626a078b898db749ebe06ab65e9"

In [189]:
def initalizeBidderContract(etherValue):
    # initialize contract
    nonce = w3.eth.get_transaction_count(my_addressB)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = {
        "chainId": chain_id, 
        "from": my_addressB, 
        "nonce": nonce,
        'to': my_addressA,
        'value' : w3.to_wei(etherValue, 'ether'),
        'gas': 2000000,
        'gasPrice': w3.eth.gas_price
        }
    
    
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyB)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # TOALGASCOST = TOTALGASCOST + tx_receipt.gasUsed

    return [tx_receipt]


In [190]:
def registrationPhaseFunctiontoInitializeSCR():
    # initialize contract
    Registration = w3.eth.contract(abi=abi, bytecode=bytecode)
    nonce = w3.eth.get_transaction_count(my_addressC)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = Registration.constructor().build_transaction(
        {"chainId": chain_id, "from": my_addressC, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyC)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    # TOALGASCOST = TOTALGASCOST + tx_receipt.gasUsed

    return [tx_receipt]

In [191]:
def registrationFunctionForSCRToStoreUID (tx_receipt, nonce,UID):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

    #  whenever we want to update something we trsact that value so gas is used for 
    # that purpose we agin perform the transaction

    call_fun = registration_sol.functions.store(UID).build_transaction(
        {"chainId": chain_id, "from": my_addressC, "nonce": w3.eth.get_transaction_count(my_addressC)}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyC)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
    # TOALGASCOST = TOTALGASCOST + tx_receipt.gasUsed

   


In [192]:
def registrationFunctionForSCRToRetrieveUID(tx_receipt):
    #certifier etrieves the submitted encrypted UID by the contract adress 
    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)
    return registration_sol.functions.retrieve().call()
    # TOALGASCOST = TOTALGASCOST + tx_receipt.gasUsed
    


In [193]:
def bidSubmissionPhaseFunctiontoInitializeSCA():
    # global TOTALASCOST
    # initialize contract
    bidSubmission = w3.eth.contract(abi=abiA, bytecode=bytecodeA)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = bidSubmission.constructor().build_transaction(
        {"chainId": chain_id, "from": my_addressA, "nonce": w3.eth.get_transaction_count(my_addressA)}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyA)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    # TOALGASCOST = TOTALGASCOST + tx_receipt.gasUsed
    return [tx_receipt]

In [194]:
def bidSubmissionFunctionForSCAToStorePB(tx_receipt,pb,flag):
    # global TOTALASCOST
    
    bid_submission_sol = w3.eth.contract(abi=abiA,address=tx_receipt.contractAddress)                     
    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction
    if flag == 'aucVar':
        call_fun = bid_submission_sol.functions.bidderEncryptedBidSubmission(pb).build_transaction({"chainId": chain_id, "from": my_addressA, "nonce": w3.eth.get_transaction_count(my_addressA)})
    else: #flag ===bidVar
        call_fun = bid_submission_sol.functions.updateBidderVariable(pb).build_transaction({"chainId": chain_id, "from": my_addressA, "nonce": w3.eth.get_transaction_count(my_addressA)})
    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyA)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
    # TOALGASCOST = TOTALGASCOST + tx_receipt.gasUsed
    return tx_call_fun_receipt


In [195]:
def bidSubmissionFunctionForSCAToRetrievePB(tx_receipt,flag):
    # global TOTALASCOST
    # calling functions in contract blockand address
    #  this line call the the contract by passing the abi and adress of the transactoin

    bid_submission_sol = w3.eth.contract(abi=abiA, address=tx_receipt.contractAddress)
    
    # just viewing something from your transaction doesnt cost anyting
    if flag == 'aucVar':
        return bid_submission_sol.functions.retrieveBidderEncryptedBidSubmission().call()
    else:
        return bid_submission_sol.functions.fetchVariable().call()



In [198]:
bidSubmissionFunctionForSCAToStorePB(values[0], '45fdgd45', 'bid')

AttributeDict({'transactionHash': HexBytes('0x0a7a1edb0eda6b8394497a2a3f2d74b838719a2679bab1af434e877bbc3bc5e7'),
 'transactionIndex': 0,
 'blockNumber': 27540,
 'blockHash': HexBytes('0x9d7716df217f6573bea265711138c13271223f156cbe776783c8d186c9691a12'),
 'from': '0xbe554a91C2e7015Ede9725eCe41228ff8b7fBC98',
 'to': '0xa0ceE090B1b16676BB9338d9D7750c620e4EB57a',
 'cumulativeGasUsed': 45087,
 'gasUsed': 45087,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status

In [199]:
abi = x
abiA = k


In [200]:
print(bidSubmissionFunctionForSCAToRetrievePB(values[0],'bid'))

45fdgd45
